## Cohort 1

## uncalibrated（scale_pos_weight=N_neg / N_pos)

In [ ]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
from youden import youden_index
import numpy as np
import warnings
warnings.filterwarnings("ignore")

model = 'catboost'
search_path = f'./Poisoning_Prediction/ML/predict_death/{model}_gridsearch_valid_test_5cv'
file = os.path.join(search_path, f'depth_5_iterations_200_learning_rate_0.05', 'all_folds_results.csv')  

save_path = './Poisoning_Prediction/prediction_plot/model_evaluate_youden/death/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

def demo():
    """
    demo for youden index
    """
    np.random.seed(42)
    y_true = np.random.randint(0, 2, size=50)
    y_score = np.random.rand(50)
    # df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, step=5)
    df, mj_val, mf1_val, auc = youden_index(y_true, y_score, thr_start=0, thr_end=0.6, thr_step=0.005)
    print('-' * 100)
    dft = pd.DataFrame()
    for col in df.columns:
        dft[col] = df[col].map("{:.3f}".format)
    print(dft)
    print('-' * 100)
    print('max_youden_value: {:.3f}'.format(mj_val))
    print('max_F1_value: {:.3f}'.format(mf1_val))
    print('AUC: {:.3f}'.format(auc))
    mj_idx = df['YoudenIdx'].idxmax()
    thr_val = df['Thr'][mj_idx]
    print('The max youden index correspond to threshold is {:.3f}'.format(thr_val))
    mf1_idx = df['F1'].idxmax()
    thr_val = df['Thr'][mf1_idx]
    print('The max F1 correspond to threshold is {:.3f}'.format(thr_val))


def prediction_model_evaluate(file):
    """
    Final Catboost model youden index
    """
    y_true_pred = pd.read_csv(file)
    y_true =y_true_pred['y_test']
    y_score = y_true_pred['y_pred']
    # youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, step=1) ## 阈值每次增加0.01
    youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, thr_start=0, thr_end=0.6, thr_step=0.005)
    # print(youden_df)

    print('-' * 100)
    youden_dft = pd.DataFrame()
    for col in youden_df.columns:
        youden_dft[col] = youden_df[col].map("{:.3f}".format)
    print(youden_dft)
    print('-' * 100)
    print('max_youden_value: {:.3f}'.format(mj_val))
    print('max_F1_value: {:.3f}'.format(mf1_val))
    print('AUC: {:.3f}'.format(auc))

    mj_idx = youden_df['YoudenIdx'].idxmax()
    thr_val = youden_df['Thr'][mj_idx]
    print('The max youden index correspond to threshold is {:.3f}'.format(thr_val))

    mf1_idx = youden_df['F1'].idxmax()
    thr_val = youden_df['Thr'][mf1_idx]
    print('The max F1 correspond to threshold is {:.3f}'.format(thr_val))
    # print(y_true_pred[y_true_pred['y_tepred_scaled']>=thr_val])
    youden_dft.to_csv(save_path + 'youden index_5-folds_final_model.csv')
    youden_dft.to_excel(save_path + 'youden index_5-folds_final_model.xlsx',index=False)

if __name__ == '__main__':
    # demo()
    prediction_model_evaluate(file)


----------------------------------------------------------------------------------------------------
       Thr    ACC    PPV    NPV Sens(Rec/TPR)   Spec YoudenIdx     F1  \
0    0.000  0.084  0.084  0.000         1.000  0.000     0.000  0.156   
1    0.005  0.084  0.084  0.000         1.000  0.000     0.000  0.156   
2    0.010  0.084  0.084  0.000         1.000  0.000     0.000  0.156   
3    0.015  0.084  0.084  0.000         1.000  0.000     0.000  0.156   
4    0.020  0.084  0.084  0.000         1.000  0.000     0.000  0.156   
..     ...    ...    ...    ...           ...    ...       ...    ...   
116  0.580  0.910  0.463  0.944         0.378  0.960     0.338  0.416   
117  0.585  0.908  0.441  0.939         0.317  0.963     0.280  0.369   
118  0.590  0.910  0.455  0.938         0.305  0.966     0.271  0.365   
119  0.595  0.912  0.472  0.938         0.305  0.969     0.273  0.370   
120  0.600  0.913  0.481  0.938         0.305  0.970     0.275  0.373   

    TruePositives True

## Platt Scaling

In [ ]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
from youden import youden_index
import numpy as np
import warnings
warnings.filterwarnings("ignore")

model = 'catboost'
search_path = f'./Poisoning_Prediction/ML/predict_death_calibration/{model}/'
file = os.path.join(search_path, f'depth_5_iterations_200_learning_rate_0.05', 'all_folds_results.csv')

save_path = './Poisoning_Prediction/prediction_plot/model_evaluate_youden/death_calibration/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

def demo():
    """
    demo for youden index
    """
    np.random.seed(42)
    y_true = np.random.randint(0, 2, size=50)
    y_score = np.random.rand(50)

    # df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, step=5)
    df, mj_val, mf1_val, auc = youden_index(y_true, y_score, thr_start=0, thr_end=0.6, thr_step=0.005)
    print('-' * 100)

    dft = pd.DataFrame()
    for col in df.columns:
        dft[col] = df[col].map("{:.3f}".format)
    print(dft)

    print('-' * 100)
    print('max_youden_value: {:.3f}'.format(mj_val))
    print('max_F1_value: {:.3f}'.format(mf1_val))
    print('AUC: {:.3f}'.format(auc))

    mj_idx = df['YoudenIdx'].idxmax()
    thr_val = df['Thr'][mj_idx]
    print('The max youden index correspond to threshold is {:.3f}'.format(thr_val))

    mf1_idx = df['F1'].idxmax()
    thr_val = df['Thr'][mf1_idx]
    print('The max F1 correspond to threshold is {:.3f}'.format(thr_val))

def prediction_model_evaluate(file):
    """
    Final Catboost model youden index
    """
    
    y_true_pred = pd.read_csv(file)
    y_true =y_true_pred['y_test']
    y_score = y_true_pred['y_pred']
    # youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, step=1) ## 阈值每次增加0.01
    youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, thr_start=0, thr_end=0.6, thr_step=0.005)
    # print(youden_df)

    print('-' * 100)
    youden_dft = pd.DataFrame()
    for col in youden_df.columns:
        youden_dft[col] = youden_df[col].map("{:.3f}".format)
    print(youden_dft)
    print('-' * 100)
    print('max_youden_value: {:.3f}'.format(mj_val))
    print('max_F1_value: {:.3f}'.format(mf1_val))
    print('AUC: {:.3f}'.format(auc))

    mj_idx = youden_df['YoudenIdx'].idxmax()
    thr_val = youden_df['Thr'][mj_idx]
    print('The max youden index correspond to threshold is {:.3f}'.format(thr_val))

    mf1_idx = youden_df['F1'].idxmax()
    thr_val = youden_df['Thr'][mf1_idx]
    print('The max F1 correspond to threshold is {:.3f}'.format(thr_val))
    
    # print(y_true_pred[y_true_pred['y_tepred_scaled']>=thr_val])


    youden_dft.to_csv(save_path + 'youden index_5-folds_final_model.csv')
    youden_dft.to_excel(save_path + 'youden index_5-folds_final_model.xlsx',index=False)

if __name__ == '__main__':
    # demo()
    prediction_model_evaluate(file)


----------------------------------------------------------------------------------------------------
       Thr    ACC    PPV    NPV Sens(Rec/TPR)   Spec YoudenIdx     F1  \
0    0.000  0.084  0.084  0.000         1.000  0.000     0.000  0.156   
1    0.005  0.181  0.092  0.980         0.976  0.108     0.084  0.168   
2    0.010  0.257  0.099  0.983         0.963  0.192     0.156  0.180   
3    0.015  0.320  0.107  0.987         0.963  0.261     0.224  0.193   
4    0.020  0.400  0.118  0.984         0.939  0.350     0.289  0.209   
..     ...    ...    ...    ...           ...    ...       ...    ...   
116  0.580  0.926  0.708  0.931         0.207  0.992     0.199  0.321   
117  0.585  0.925  0.696  0.930         0.195  0.992     0.187  0.305   
118  0.590  0.925  0.696  0.930         0.195  0.992     0.187  0.305   
119  0.595  0.924  0.682  0.929         0.183  0.992     0.175  0.288   
120  0.600  0.923  0.667  0.928         0.171  0.992     0.163  0.272   

    TruePositives True

## Cohort 2

## uncalibrated（scale_pos_weight=N_neg / N_pos)

In [ ]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
from youden import youden_index
import numpy as np
import warnings
warnings.filterwarnings("ignore")

model = 'catboost'
search_path = f'./Poisoning_Prediction/ML/predict_non-recovery/{model}_fixed_valid_test_5cv'
file = os.path.join(search_path, f'depth_5_iterations_200_learning_rate_0.05', 'all_folds_results.csv')  ## 训练模型时试随机种子

save_path = './Poisoning_Prediction/prediction_plot/model_evaluate_youden/non-recovery/'

if not os.path.exists(save_path):
    os.makedirs(save_path)


def demo():
    """
    demo for youden index
    """
    np.random.seed(42)
    y_true = np.random.randint(0, 2, size=50)
    y_score = np.random.rand(50)
    # df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, step=5)
    df, mj_val, mf1_val, auc = youden_index(y_true, y_score, thr_start=0, thr_end=0.6, thr_step=0.005)
    print('-' * 100)
    dft = pd.DataFrame()
    for col in df.columns:
        dft[col] = df[col].map("{:.3f}".format)
    print(dft)
    print('-' * 100)
    print('max_youden_value: {:.3f}'.format(mj_val))
    print('max_F1_value: {:.3f}'.format(mf1_val))
    print('AUC: {:.3f}'.format(auc))
    mj_idx = df['YoudenIdx'].idxmax()
    thr_val = df['Thr'][mj_idx]
    print('The max youden index correspond to threshold is {:.3f}'.format(thr_val))
    mf1_idx = df['F1'].idxmax()
    thr_val = df['Thr'][mf1_idx]
    print('The max F1 correspond to threshold is {:.3f}'.format(thr_val))


def prediction_model_evaluate(file):
    """
    Final Catboost model youden index
    """
    y_true_pred = pd.read_csv(file)
    y_true =y_true_pred['y_test']
    y_score = y_true_pred['y_pred']
    # youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, step=1) ## 阈值每次增加0.01
    youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, thr_start=0, thr_end=0.6, thr_step=0.005)
    # print(youden_df)
    print('-' * 100)
    youden_dft = pd.DataFrame()
    for col in youden_df.columns:
        youden_dft[col] = youden_df[col].map("{:.3f}".format)
    print(youden_dft)
    print('-' * 100)
    print('max_youden_value: {:.3f}'.format(mj_val))
    print('max_F1_value: {:.3f}'.format(mf1_val))
    print('AUC: {:.3f}'.format(auc))
    mj_idx = youden_df['YoudenIdx'].idxmax()
    thr_val = youden_df['Thr'][mj_idx]
    print('The max youden index correspond to threshold is {:.3f}'.format(thr_val))
    mf1_idx = youden_df['F1'].idxmax()
    thr_val = youden_df['Thr'][mf1_idx]
    print('The max F1 correspond to threshold is {:.3f}'.format(thr_val))
    # print(y_true_pred[y_true_pred['y_tepred_scaled']>=thr_val])
    youden_dft.to_csv(save_path + 'youden index_5-folds_final_model.csv')
    youden_dft.to_excel(save_path + 'youden index_5-folds_final_model.xlsx',index=False)

if __name__ == '__main__':
    # demo()
    prediction_model_evaluate(file)


----------------------------------------------------------------------------------------------------
       Thr    ACC    PPV    NPV Sens(Rec/TPR)   Spec YoudenIdx     F1  \
0    0.000  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
1    0.005  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
2    0.010  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
3    0.015  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
4    0.020  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
..     ...    ...    ...    ...           ...    ...       ...    ...   
116  0.580  0.818  0.662  0.857         0.537  0.910     0.447  0.593   
117  0.585  0.822  0.681  0.855         0.525  0.919     0.444  0.593   
118  0.590  0.826  0.701  0.854         0.517  0.927     0.444  0.595   
119  0.595  0.830  0.719  0.854         0.512  0.934     0.447  0.599   
120  0.600  0.832  0.736  0.851         0.500  0.941     0.441  0.596   

    TruePositives True

## Platt Scaling

In [ ]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
from youden import youden_index
import numpy as np
import warnings
warnings.filterwarnings("ignore")

model = 'catboost'
search_path = f'./Poisoning_Prediction/ML/predict_non-recovery_calibration/{model}/'
file = os.path.join(search_path, f'depth_5_iterations_200_learning_rate_0.05', 'all_folds_results.csv')

save_path = './Poisoning_Prediction/prediction_plot/model_evaluate_youden/non-recovery_calibration/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

def demo():
    """
    demo for youden index
    """
    np.random.seed(42)
    y_true = np.random.randint(0, 2, size=50)
    y_score = np.random.rand(50)
    # df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, step=5)
    df, mj_val, mf1_val, auc = youden_index(y_true, y_score, thr_start=0, thr_end=0.6, thr_step=0.005)
    print('-' * 100)
    dft = pd.DataFrame()
    for col in df.columns:
        dft[col] = df[col].map("{:.3f}".format)
    print(dft)
    print('-' * 100)
    print('max_youden_value: {:.3f}'.format(mj_val))
    print('max_F1_value: {:.3f}'.format(mf1_val))
    print('AUC: {:.3f}'.format(auc))
    mj_idx = df['YoudenIdx'].idxmax()
    thr_val = df['Thr'][mj_idx]
    print('The max youden index correspond to threshold is {:.3f}'.format(thr_val))
    mf1_idx = df['F1'].idxmax()
    thr_val = df['Thr'][mf1_idx]
    print('The max F1 correspond to threshold is {:.3f}'.format(thr_val))


def prediction_model_evaluate(file):
    """
    Final Catboost model youden index
    """
    y_true_pred = pd.read_csv(file)
    y_true =y_true_pred['y_test']
    y_score = y_true_pred['y_pred']
    # youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, step=1) 
    youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, thr_start=0, thr_end=0.6, thr_step=0.005)
    # print(youden_df)
    print('-' * 100)
    youden_dft = pd.DataFrame()
    for col in youden_df.columns:
        youden_dft[col] = youden_df[col].map("{:.3f}".format)
    print(youden_dft)
    print('-' * 100)
    print('max_youden_value: {:.3f}'.format(mj_val))
    print('max_F1_value: {:.3f}'.format(mf1_val))
    print('AUC: {:.3f}'.format(auc))
    mj_idx = youden_df['YoudenIdx'].idxmax()
    thr_val = youden_df['Thr'][mj_idx]
    print('The max youden index correspond to threshold is {:.3f}'.format(thr_val))
    mf1_idx = youden_df['F1'].idxmax()
    thr_val = youden_df['Thr'][mf1_idx]
    print('The max F1 correspond to threshold is {:.3f}'.format(thr_val))
    
    # print(y_true_pred[y_true_pred['y_tepred_scaled']>=thr_val])
    youden_dft.to_csv(save_path + 'youden index_5-folds_final_model.csv')
    youden_dft.to_excel(save_path + 'youden index_5-folds_final_model.xlsx',index=False)

if __name__ == '__main__':
    # demo()
    prediction_model_evaluate(file)
    


----------------------------------------------------------------------------------------------------
       Thr    ACC    PPV    NPV Sens(Rec/TPR)   Spec YoudenIdx     F1  \
0    0.000  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
1    0.005  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
2    0.010  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
3    0.015  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
4    0.020  0.247  0.247  0.000         1.000  0.000     0.000  0.396   
..     ...    ...    ...    ...           ...    ...       ...    ...   
116  0.580  0.837  0.831  0.838         0.429  0.971     0.400  0.566   
117  0.585  0.836  0.829  0.837         0.425  0.971     0.396  0.562   
118  0.590  0.835  0.828  0.836         0.421  0.971     0.392  0.558   
119  0.595  0.834  0.826  0.835         0.417  0.971     0.388  0.554   
120  0.600  0.834  0.826  0.835         0.417  0.971     0.388  0.554   

    TruePositives True